In [1]:
import datetime
import random
import math
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal

%load_ext line_profiler


In [2]:
def make_5min_bars(tick_data):
    #Resample to get 5min bars
    five_min_data = pd.DataFrame(
        tick_data['last'].resample('5Min', loffset=datetime.timedelta(minutes=5)).ohlc())
        
    #Create RTH Calendar
        
    #We hack the NYSE Calendar extending the close until 4:15
    class CMERTHCalendar(mcal.exchange_calendar_nyse.NYSEExchangeCalendar):
        @property
        def close_time(self):
            return datetime.time(16, 15)
        
    nyse = CMERTHCalendar()
    print(nyse.tz.zone)
    schedule = nyse.schedule(start_date=five_min_data.index.min(), 
                             end_date=five_min_data.index.max())
        
    #Filter out those bars that occur during RTH
    five_min_data['dates'] = pd.to_datetime(five_min_data.index.to_datetime().date)
    five_min_data['valid_date'] = five_min_data['dates'].isin(schedule.index)
    five_min_data['valid_time'] = False
    during_rth = five_min_data['valid_date'] & \
        (five_min_data.index > schedule.loc[five_min_data['dates'],'market_open']) & \
        (five_min_data.index <= schedule.loc[five_min_data['dates'],'market_close'])
    five_min_data.loc[during_rth, 'valid_time'] = True
    five_min_data = five_min_data[five_min_data['valid_time'] == True]
    five_min_data.drop(['dates','valid_date','valid_time'], axis=1, inplace=True)
        
    #Add ema
    five_min_data['ema'] = five_min_data['close'].ewm(span=20, min_periods=20).mean()

    #Reset index
    five_min_data.reset_index(inplace=True)
        
    #Add column for number of seconds elapsed in trading day
    five_min_data['sec'] = (five_min_data['date'].values 
                            - five_min_data['date'].values.astype('datetime64[D]')) / np.timedelta64(1,'s')

    #Calculate sin & cos time
    #24hr time is a cyclical continuous feature
    seconds_in_day = 24*60*60
    five_min_data['sin_time'] = np.sin(2*np.pi*five_min_data['sec']/seconds_in_day)
    five_min_data['cos_time'] = np.cos(2*np.pi*five_min_data['sec']/seconds_in_day)

    five_min_data.drop('sec', axis=1, inplace=True)
        
    return five_min_data

In [17]:
#td = pd.read_feather('../data/processed/ES_tick.feather')
#td = td[td['date'] > '2017-07-29']
#td = td[td['date'] > '2018-01-24']
#Create Index from date column
#td.index = td['date']
#td.drop(labels=['date'],axis=1,inplace=True)
#Remove duplicates -- timestamps are equal!!!
#td = td[~td.index.duplicated()]
#td.head()
td = pd.read_hdf('../data/processed/store.h5', key='tick_data')
td.columns = ['last','volume']
td.index.rename('date', inplace=True)
#td.index = td.index.tz_localize('US/Eastern')
td = td[~td.index.duplicated()]
td.head()

,last,volume
date,,
2006-01-02 17:00:02,1255.25,2
2006-01-02 17:00:04,1255.50,3
2006-01-02 17:00:05,1255.25,1
2006-01-02 17:00:08,1255.50,5
2006-01-02 17:00:09,1255.50,5


In [20]:
td.index = td.index.tz_localize('America/Chicago')
td.head()

,last,volume
date,,
2006-01-02 17:00:02-06:00,1255.25,2
2006-01-02 17:00:04-06:00,1255.50,3
2006-01-02 17:00:05-06:00,1255.25,1
2006-01-02 17:00:08-06:00,1255.50,5
2006-01-02 17:00:09-06:00,1255.50,5


In [22]:
td.index = td.index.tz_convert('America/New_York')
td.head()

,last,volume
date,,
2006-01-02 18:00:02-05:00,1255.25,2
2006-01-02 18:00:04-05:00,1255.50,3
2006-01-02 18:00:05-05:00,1255.25,1
2006-01-02 18:00:08-05:00,1255.50,5
2006-01-02 18:00:09-05:00,1255.50,5


In [23]:
store = pd.HDFStore('../data/processed/store.h5')
store.put('processed_data',td)
store.close()

In [3]:
td = pd.read_hdf('../data/processed/store.h5', key='processed_data')
td.head()

,last,volume
date,,
2006-01-02 17:00:02-05:00,1255.25,2
2006-01-02 17:00:04-05:00,1255.50,3
2006-01-02 17:00:05-05:00,1255.25,1
2006-01-02 17:00:08-05:00,1255.50,5
2006-01-02 17:00:09-05:00,1255.50,5


In [24]:
td.shape

(192917901, 2)

In [25]:
fd = make_5min_bars(td)

#Add columns tracking reward for buy/sell actions and number of bars until reward
fd.loc[:,'sell_r'] = 0
fd.loc[:,'sell_b'] = 0
fd.loc[:,'buy_r'] = 0
fd.loc[:,'buy_b'] = 0

America/New_York


In [29]:
fd['close'].isnull().values.any()

False

In [30]:
fd.tail(100)

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b
253226,2018-06-21 14:45:00-04:00,2756.75,2757.25,2755.75,2756.50,2757.807597,-0.980785,1.950903e-01,0,0,0,0
253227,2018-06-21 14:50:00-04:00,2756.50,2757.75,2756.25,2757.75,2757.802112,-0.976296,2.164396e-01,0,0,0,0
253228,2018-06-21 14:55:00-04:00,2757.50,2757.75,2755.50,2755.75,2757.606673,-0.971342,2.376859e-01,0,0,0,0
253229,2018-06-21 15:00:00-04:00,2755.75,2756.25,2754.25,2754.25,2757.286990,-0.965926,2.588190e-01,0,0,0,0
253230,2018-06-21 15:05:00-04:00,2754.25,2755.50,2753.00,2755.25,2757.092991,-0.960050,2.798290e-01,0,0,0,0
253231,2018-06-21 15:10:00-04:00,2755.00,2757.00,2753.75,2756.00,2756.988896,-0.953717,3.007058e-01,0,0,0,0
253232,2018-06-21 15:15:00-04:00,2756.00,2756.25,2753.25,2754.25,2756.728049,-0.946930,3.214395e-01,0,0,0,0
253233,2018-06-21 15:20:00-04:00,2754.25,2754.25,2751.25,2752.00,2756.277759,-0.939693,3.420201e-01,0,0,0,0
253234,2018-06-21 15:25:00-04:00,2752.25,2753.25,2747.75,2752.50,2755.917972,-0.932008,3.624380e-01,0,0,0,0
253235,2018-06-21 15:30:00-04:00,2752.25,2755.00,2751.50,2753.75,2755.711499,-0.923880,3.826834e-01,0,0,0,0


In [31]:
def simulate(start_index, action, order_price):
    position = 'flat'
    working_order = None
    reward = 0
    #for i, r in fd[start_index+1:].iterrows():
    for i in fd[start_index+1:start_index+82].index:
         # Get tick data for the current bar
        #ts_end = r['date']
        ts_end = fd.at[i, 'date']
        ts_start = ts_end - pd.Timedelta(minutes=5)
        #mask1 = td.index > ts_start
        #mask2 = td.index <= ts_end
        #mask = mask1 & mask2
        #ticks = td[mask]
        ticks = td.loc[ts_start:ts_end, :]
        if position == 'flat':
            if action == 'buy':
                working_order = 'buy'
                target_price = order_price + 1
                stop_price = order_price - 1
            elif action == 'sell':
                working_order = 'sell'
                target_price = order_price - 1
                stop_price = order_price + 1
                
        
        #for t_i, t_r in ticks.iterrows():
        for t_i in ticks.index:
            price = ticks.at[t_i, 'last']
            if position == 'flat':
                if working_order == 'buy':
                    if price < order_price:
                        position = 'long'
                        working_order = None
                        order_price = 0
                elif working_order == 'sell':
                    if price > order_price:
                        position = 'short'
                        working_order = None
                        order_price = 0
            elif position == 'long':
                if price > target_price:
                    position = 'flat'
                    reward = 50
                elif price <= stop_price:
                    position = 'flat'
                    reward = -50
            elif position == 'short':
                if price < target_price:
                    position = 'flat'
                    reward = 50
                elif price >= stop_price:
                    position = 'flat'
                    reward = -50
        
        if position == 'flat':
            break
            
    return reward, i - (start_index+1)
    

In [18]:
%lprun -f simulate simulate(0, 'buy', fd.loc[0, 'low'])

In [12]:
fd.shape

(253323, 12)

In [15]:
td.index.dtype

datetime64[ns, US/Eastern]

In [32]:
%%time
for index, row in fd[:-1].iterrows():
    fd.loc[index, 'buy_r'], fd.loc[index, 'buy_b'] = simulate(index, 'buy', row['low'])
    fd.loc[index, 'sell_r'], fd.loc[index, 'sell_b'] = simulate(index, 'sell', row['high'])

CPU times: user 1h 32min 34s, sys: 28 ms, total: 1h 32min 34s
Wall time: 1h 32min 33s


In [7]:
%%time
for index in fd[:-1].index:
    fd.loc[index, 'buy_r'], fd.loc[index, 'buy_b'] = simulate(index, 'buy', fd.at[index,'low'])
    fd.loc[index, 'sell_r'], fd.loc[index, 'sell_b'] = simulate(index, 'sell', fd.at[index,'high'])

CPU times: user 20min 14s, sys: 2min 32s, total: 22min 47s
Wall time: 22min 47s


In [21]:
fd.tail(100)

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b
253223,2018-06-21 14:30:00-04:00,2752.25,2755.00,2751.50,2753.75,2755.716893,-0.991445,1.305262e-01,0,0,50,0
253224,2018-06-21 14:35:00-04:00,2753.50,2753.75,2748.75,2749.25,2755.100496,-0.988362,1.521234e-01,0,0,50,0
253225,2018-06-21 14:40:00-04:00,2749.25,2750.50,2747.75,2750.25,2754.638204,-0.984808,1.736482e-01,0,0,50,0
253226,2018-06-21 14:45:00-04:00,2750.00,2750.00,2747.25,2747.75,2753.981747,-0.980785,1.950903e-01,-50,0,50,0
253227,2018-06-21 14:50:00-04:00,2747.75,2752.00,2747.00,2752.00,2753.792895,-0.976296,2.164396e-01,-50,0,0,0
253228,2018-06-21 14:55:00-04:00,2752.00,2754.00,2750.25,2752.25,2753.645872,-0.971342,2.376859e-01,50,0,0,0
253229,2018-06-21 15:00:00-04:00,2752.50,2754.25,2751.75,2753.00,2753.584330,-0.965926,2.588190e-01,0,0,50,1
253230,2018-06-21 15:05:00-04:00,2752.75,2754.00,2751.50,2752.25,2753.457195,-0.960050,2.798290e-01,0,0,0,0
253231,2018-06-21 15:10:00-04:00,2752.25,2753.50,2752.00,2752.75,2753.389815,-0.953717,3.007058e-01,50,0,-50,6
253232,2018-06-21 15:15:00-04:00,2753.00,2754.00,2751.75,2752.00,2753.257404,-0.946930,3.214395e-01,0,0,0,0


In [33]:
store = pd.HDFStore('../data/processed/store.h5')
store.put('int_data',fd)
store.close()

In [60]:
fd.to_feather('../data/processed/ES_5mintrading.feather')

In [13]:
fd = pd.read_feather('../data/processed/ES_5mintrading.feather')
fd.tail()

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b,cdl_body,cdl_ut,cdl_lt,cdl_rng
10000,2018-01-25 15:55:00-05:00,2838.50,2840.50,2838.25,2838.50,2838.062309,-0.722364,0.691513,-50,3,0,0,0.00,2.00,0.25,2.25
10001,2018-01-25 16:00:00-05:00,2838.75,2840.75,2838.25,2839.75,2838.223042,-0.707107,0.707107,-50,2,0,0,1.00,1.00,0.50,2.50
10002,2018-01-25 16:05:00-05:00,2840.00,2841.00,2839.50,2840.00,2838.392276,-0.691513,0.722364,0,0,0,0,0.00,1.00,0.50,1.50
10003,2018-01-25 16:10:00-05:00,2840.25,2840.75,2840.00,2840.75,2838.616821,-0.675590,0.737277,-50,0,0,0,0.50,0.00,0.25,0.75
10004,2018-01-25 16:15:00-05:00,2840.75,2841.75,2840.50,2841.00,2838.843790,-0.659346,0.751840,0,0,0,0,0.25,0.75,0.25,1.25


In [34]:
fd['cdl_sign'] = np.sign(fd['close'] - fd['open'])
fd['cdl_body'] = np.absolute(fd['close'] - fd['open'])
#fd['cdl_ut'] = fd['high'] - fd['close'] if fd['cdl_body'] > 0 else fd['high'] - fd['open']
fd['cdl_ut'] = np.where(fd['cdl_sign'] > 0, fd['high'] - fd['close'], fd['high'] - fd['open'])
fd['cdl_lt'] = np.where(fd['cdl_sign'] > 0, fd['open'] - fd['low'], fd['close'] - fd['low'])
fd['cdl_rng'] = fd['high'] - fd['low']
fd['cdl_hl'] = np.where(fd['low'] >= fd['low'].shift(), 1, 0) #higher low
fd['cdl_lh'] = np.where(fd['high'] <= fd['high'].shift(), 1, 0) #lower high
fd.tail(10)

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b,cdl_sign,cdl_body,cdl_ut,cdl_lt,cdl_rng,cdl_hl,cdl_lh
253316,2018-06-22 15:30:00-04:00,2764.00,2765.00,2763.75,2764.00,2764.390244,-0.923880,0.382683,0,0,-50,0,0.0,0.00,1.00,0.25,1.25,1,0
253317,2018-06-22 15:35:00-04:00,2764.00,2764.00,2762.50,2763.75,2764.329269,-0.915311,0.402747,0,0,0,0,-1.0,0.25,0.00,1.25,1.50,0,1
253318,2018-06-22 15:40:00-04:00,2763.50,2763.75,2762.50,2762.50,2764.155053,-0.906308,0.422618,0,0,50,1,-1.0,1.00,0.25,0.00,1.25,1,1
253319,2018-06-22 15:45:00-04:00,2762.75,2763.50,2762.25,2762.25,2763.973619,-0.896873,0.442289,-50,0,0,0,-1.0,0.50,0.75,0.00,1.25,0,1
253320,2018-06-22 15:50:00-04:00,2762.50,2765.00,2762.25,2763.50,2763.928513,-0.887011,0.461749,0,0,-50,1,1.0,1.00,1.50,0.25,2.75,1,0
253321,2018-06-22 15:55:00-04:00,2763.25,2764.50,2762.00,2762.50,2763.792464,-0.876727,0.480989,0,0,-50,0,-1.0,0.75,1.25,0.50,2.50,0,1
253322,2018-06-22 16:00:00-04:00,2762.25,2762.75,2755.25,2757.50,2763.193181,-0.866025,0.500000,0,0,0,0,-1.0,4.75,0.50,2.25,7.50,0,1
253323,2018-06-22 16:05:00-04:00,2757.75,2761.75,2757.50,2761.00,2762.984307,-0.854912,0.518773,0,0,0,0,1.0,3.25,0.75,0.25,4.25,1,1
253324,2018-06-22 16:10:00-04:00,2761.25,2761.50,2759.25,2759.50,2762.652468,-0.843391,0.537300,0,0,0,0,-1.0,1.75,0.25,0.25,2.25,1,1
253325,2018-06-22 16:15:00-04:00,2759.50,2760.75,2758.75,2759.50,2762.352233,-0.831470,0.555570,0,0,0,0,0.0,0.00,1.25,0.75,2.00,0,1


In [35]:
fd.shape

(253326, 19)

In [36]:
store = pd.HDFStore('../data/processed/store.h5')
store.put('final_data',fd)
store.close()

In [25]:
fd.to_feather('../data/processed/ES_5mintrading.feather')

In [26]:
td = pd.read_feather('../data/processed/ES_5mintrading.feather')
td.tail()

,date,open,high,low,close,ema,sin_time,cos_time,sell_r,sell_b,buy_r,buy_b,cdl_body,cdl_ut,cdl_lt,cdl_rng,cdl_sign,cdl_hl,cdl_lh
10000,2018-01-25 15:55:00-05:00,2838.50,2840.50,2838.25,2838.50,2838.062309,-0.722364,0.691513,-50,3,0,0,0.00,2.00,0.25,2.25,0.0,1,0
10001,2018-01-25 16:00:00-05:00,2838.75,2840.75,2838.25,2839.75,2838.223042,-0.707107,0.707107,-50,2,0,0,1.00,1.00,0.50,2.50,1.0,1,0
10002,2018-01-25 16:05:00-05:00,2840.00,2841.00,2839.50,2840.00,2838.392276,-0.691513,0.722364,0,0,0,0,0.00,1.00,0.50,1.50,0.0,1,0
10003,2018-01-25 16:10:00-05:00,2840.25,2840.75,2840.00,2840.75,2838.616821,-0.675590,0.737277,-50,0,0,0,0.50,0.00,0.25,0.75,1.0,1,1
10004,2018-01-25 16:15:00-05:00,2840.75,2841.75,2840.50,2841.00,2838.843790,-0.659346,0.751840,0,0,0,0,0.25,0.75,0.25,1.25,1.0,1,0
